In [1]:
from pathlib import Path
from time import time
import math
import numpy as np

input_file = Path('../../AdventOfCode_inputs/AoC-2025-12-input.txt')

rinput = input_file.read_text()

In [2]:
sinput = '''0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
'''

In [3]:
ainput = rinput

In [4]:
[*shapes, trees] = ainput.split('\n\n')
shapes = [ s.splitlines()[1:]  for s in shapes]
trees = [ [list(map(int,a[0].split("x"))), list(map(int, a[1].split())) ] for a in [t.split(': ') for t in trees.splitlines()]]

In [5]:
svols = [sum( shapes[j][i].count('#') for i in range(3)) for j in range(6)]

In [6]:
def shape2Num(shape):
    return eval("0b"+"".join([ '1' if c=="#" else '0' for c in "".join(shape) ]))
def flipshape(shape):
    return list(s[::-1] for s in shape)
def rotateshape(shape):
    transp = np.array([list(r) for r in shape]).transpose()
    return [ "".join(transp[i])[::-1] for i in range(3) ]
def allshapeNums(shape):
    r = [0]*8
    r[0] = shape
    r[1] = rotateshape(r[0])
    r[2] = rotateshape(r[1])
    r[3] = rotateshape(r[2])
    r[4] = flipshape(r[0])
    r[5] = flipshape(r[1])
    r[6] = flipshape(r[2])
    r[7] = flipshape(r[3])
    return set(map(shape2Num, r))
def num2Shape(num):
     return [("..."+s[2:])[-3:].replace('1','#').replace('0','.') for s in [bin(num//2**6%2**3), bin(num//2**3%2**3), bin(num%2**3)]]

Slist = [ allshapeNums(s) for s in shapes ]
S = sorted(list(set.union(*Slist)))

masks = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,0), (0,1), (1,-1), (1,0), (1,1)]

In [7]:
S

[94,
 95,
 127,
 223,
 244,
 247,
 253,
 307,
 311,
 319,
 367,
 379,
 381,
 382,
 409,
 415,
 439,
 445,
 463,
 471,
 473,
 475,
 478,
 487,
 493,
 499,
 500,
 502,
 505,
 508]

In [8]:
def llt(a,b):
    for i,j in zip(a,b):
        if i>=j:
            return False
    return True
def lle(a,b):
    for i,j in zip(a,b):
        if i>j:
            return False
    return True

**Part One**

In [9]:
from pulp import *
from functools import cmp_to_key

SLV = None #pulp.PULP_CBC_CMD(msg=False, warmStart=True)

startTime = time()
count = 0
cache = {}

st = 0
ed = len(trees)
tmp = trees[st:ed]
tmp.sort(key=cmp_to_key(lle))
trees[st:ed] = tmp

for t in range(st,ed):

    h, w = trees[t][0]
    
    # before the hard stuff, try the easy stuff
    if h*w >= 9*sum(trees[t][1]) :
        # every present can sit in a 3x3 box, no packing needed
        print("Trivially large: ", t, trees[t])
        count += 1
        continue
    elif h*w < sum( svols[i]*trees[t][1][i] for i in range(6)):
        # even if you could take the presents appart, then wouldn't fit
        print("Trivially small: ", t, trees[t])
        continue
    else:
        print("Not trivial: ", t, trees[t], "\n\n\n\n")
    
    solve = LpProblem("TreeProblem-%s"%t)
    
    # variables: T[x,y,s] = 1 <==> there is a tile s at pos x,y
    #T = {(i,j,s):Int("T__%s,%s,%s" % (i,j,s)) for s in S for j in range(w) for i in range(h) }
    T = LpVariable.dicts("T", (range(-1,h+1),range(-1,w+1),S), cat="Binary")

    # find old sol for warm start
    best = (0,0,0,0,0,0,0,0)
    for k in cache.keys():
        # grid is smaller or equal, and number of presents is better
        if lle(k[:2], trees[t][0]) and lle(best[2:], k[2:]):
            best = k
            
    # warm start the solver
    if best != (0,0,0,0,0,0,0,0):
        print("\n\n\n\n#########################\n", t, "Warm START:",trees[t], "from", best, "\n#########################\n\n\n\n")
        oldsol = cache[best]
        for k in oldsol:
            T[k[0]][k[1]][k[2]].setInitialValue(1)
    
    # Tiles cannot be positioned on the edges
    for i in range(-1,h+1):
        for s in S:
            T[i][-1][s] = 0
            T[i][0][s] = 0
            T[i][w-1][s] = 0
            T[i][w][s] = 0
    for j in range(-1,w+1):
        for s in S:
            T[-1][j][s] = 0
            T[0][j][s] = 0
            T[h-1][j][s] = 0
            T[h][j][s] = 0

    # only one s at each x,y
    for i in range(h):
        for j in range(w):
            solve+=(lpSum(T[i][j][s] for s in S)<=1)

    # exactly the correct number of each s
    for k in range(len(shapes)):
        solve+=(lpSum(lpSum(T[i][j][s] for i in range(h) for j in range(w)) for s in Slist[k]) == trees[t][1][k])

    # no overlaps
    for i in range(h):
        for j in range(w):
            solve+=( lpSum( lpSum(T[i+masks[k][0]][j+masks[k][1]][s]*((s//2**k)%2) for s in S) for k in range(9)) <= 1 )
    
    solve.solve(SLV)
    if solve.status > 0:
        count+=1
        # cache solution
        sol = []
        for i in range(1,h-1):
            for j in range(1,w-1):
                for s in S:
                    if T[i][j][s].value() != 0.0 :
                        sol.append((i,j,s))
        cache[(trees[t][0][0],trees[t][0][1],
               trees[t][1][0],trees[t][1][1],trees[t][1][2],trees[t][1][5],trees[t][1][4],trees[t][1][5])] = sol

    print("\n\n\n\n#########################\n", t, count, "\n#########################\n\n\n\n")

ans1 = count

print("%s s"%(time()-startTime))  
ans1


Trivially large:  0 [[41, 48], [47, 27, 36, 34, 31, 32]]
Trivially small:  1 [[36, 50], [46, 53, 35, 41, 46, 53]]
Trivially small:  2 [[43, 42], [39, 45, 50, 52, 48, 44]]
Trivially small:  3 [[46, 44], [57, 49, 51, 49, 57, 49]]
Trivially small:  4 [[39, 49], [55, 45, 48, 55, 48, 44]]
Trivially large:  5 [[45, 44], [39, 38, 37, 30, 36, 30]]
Trivially small:  6 [[41, 42], [56, 42, 45, 38, 44, 42]]
Trivially small:  7 [[40, 50], [47, 52, 64, 50, 44, 54]]
Trivially small:  8 [[47, 38], [56, 50, 58, 37, 42, 37]]
Trivially large:  9 [[49, 45], [35, 45, 39, 50, 39, 31]]
Trivially large:  10 [[35, 38], [26, 22, 19, 19, 31, 15]]
Trivially large:  11 [[35, 44], [24, 29, 40, 17, 20, 24]]
Trivially small:  12 [[39, 39], [37, 38, 39, 42, 36, 42]]
Trivially large:  13 [[37, 41], [27, 22, 28, 22, 29, 28]]
Trivially large:  14 [[42, 48], [41, 47, 37, 32, 35, 31]]
Trivially small:  15 [[39, 36], [27, 39, 39, 37, 33, 41]]
Trivially large:  16 [[36, 41], [25, 28, 21, 32, 26, 23]]
Trivially small:  17 [[3

492

In [10]:
ans1

492